In [85]:
# conda activate Ollama_RPG
# systemctl restart ollama


In [ ]:
!pip install ollama langchain-ollama

##### Bibliotecas e funções basicas

In [1]:
from langchain.output_parsers import PydanticOutputParser
from langchain_ollama import OllamaLLM
from langchain.prompts import PromptTemplate
from pydantic import BaseModel
from typing import List
import ollama # pega a lista de modelos que tem disponivel localmente no sistema
import json # se precisar manipular json
import html # para corrigir algumas saidas que vem com códigos de escape HTML, algo como &quot que representa " (aspas duplas).

In [2]:
models = ollama.list()
lista_modelos = []
i = 0
for model in models['models']:
    print(f"{i} - {model['model']}")
    lista_modelos.append(model['model'])
    i+=1

0 - phi3:3.8b-mini-128k-instruct-q8_0
1 - llama3.2:3b-instruct-q8_0
2 - qwen2.5:7b
3 - qwen2.5-coder:1.5b
4 - qwen2.5-coder:7b-instruct-q4_K_M
5 - nomic-embed-text:latest


In [3]:
modelo_id = 1
lista_modelos[modelo_id]

'llama3.2:3b-instruct-q8_0'

In [4]:
def model_response(system_prompt, user_prompt, schema_json, output_class):
    # Initialize Ollama
    modelo =  OllamaLLM(
        model=lista_modelos[modelo_id],
        keep_alive="20m"
    )

    # Combine prompts with format instructions
    template = """{system_prompt}
        
    {user_prompt}

    {format_instructions}

    {schema_json}
    """
    # Create parser
    parser = PydanticOutputParser(pydantic_object=output_class)

    prompt = PromptTemplate(
        template=template,
        input_variables=["system_prompt", "user_prompt", "schema_json"],
        partial_variables={"format_instructions": parser.get_format_instructions()}
    )
    
    # Generate formatted prompt
    formatted_prompt = prompt.format(
        system_prompt=system_prompt,
        user_prompt=user_prompt,
        schema_json=schema_json
    )
    
    # Get response and parse
    response = modelo.invoke(formatted_prompt)
    try:
        parsed_response = parser.parse(response)
        return parsed_response
    except Exception as e:
        print(f"Error parsing response: {e}")
        return None

In [5]:
# Função que seleciona de maneira aleatoria o que aconteceu, o que precisa ser feito e as consequencias.
import random

algo_aconteceu = ["Um exército de mortos avança", "Um tirano dominou terras", "Um mago controlou um grande mal", "Uma criatura poderosa despertou", "Um artefato sagrado foi roubado", "Um Rei bondoso foi assassinado", "Criaturas malignas estão se unindo", "Um local sagrado foi conspurcado", "Uma grande guerra se iniciou", "Um poderoso dragão surgiu"]
voce_precisa = ["Encontrar um item poderoso", "Despertar uma divindade", "Derrotar um Vilão", "Realizar uma grande viagem", "Invadir a poderosa fortaleza", "Resistir bravamente ao perigo", "Destruir algo perigoso", "Restabelecer a ordem", "Reunir um grande exército", "Resgatar o herói perdido"] 
consequencia = ["O mundo será destruído", "A escuridão tomará o mundo", "Um deus maligno despertará", "Suas terras serão tomadas", "Uma guerra eterna acontecerá", "Um reino de ódio começará", "Uma doença se espalhará", "Milhões morrerão", "O caos tomará o mundo", "A realidade irá colapsar"]



def cenario_aleatorio():
    AA = random.sample(range(len(algo_aconteceu)), 1)[0]
    VP = random.sample(range(len(voce_precisa)), 1)[0]
    SN = random.sample(range(len(consequencia)), 1)[0]
    return f"""Descrisão do Cenário:
- Gênero: Fantasy
- Algo aconteceu: {algo_aconteceu[AA]}.
- Você precisa: {voce_precisa[VP]}. 
- Senão: {consequencia[SN]}.
- Ambientação: Você é um Aventureiro que se aventura em um incrível mundo de fantasia, rastejando em perigosas masmorras e enfrentando terríveis inimigos!"""

In [6]:
# Função que seleciona de maneira aleatoria lugares, personagens e eventos.
import random

Lugares = ['Uma pequena vila', 'Um grande reino', 'Uma masmorra mortal', 'Um outro plano de existência', 'Uma passagem perigosa', 'Uma fortaleza inexpugnável', 'Um templo amaldiçoado', 'Um local ermo', 'Um local sagrado', 'Uma movimentada taverna']
Personagens = ['Um aventureiro/grupo rival', 'Uma pessoa traiçoeira', 'Um velho aventureiro', 'Um mensageiro', 'Um ser muito poderoso', 'Um exército', 'Alguém amaldiçoado', 'Um contratante misterioso', 'Um artefato inteligente', 'Uma pessoa muito rica']
Eventos = ['O Vilão surge', 'Bandidos buscam riquezas', 'Um grande mistério', 'Alguém foi morto aqui', 'Uma armadilha a frente', 'Um ataque surpresa', 'Os servos do Vilão estão aqui', 'Uma passagem difícil/secreta', 'Monstros errantes aparecem', 'O Vilão ataca']

def cenas_aleatorias():
    L = random.sample(range(len(Lugares)), 4)
    P = random.sample(range(len(Personagens)), 4)
    E = random.sample(range(len(Eventos)), 4)
    lugar = f"{Lugares[L[0]]}, {Lugares[L[1]]}, {Lugares[L[2]]}, {Lugares[L[3]]}"
    personagens = f"{Personagens[P[0]]}, {Personagens[P[1]]}, {Personagens[P[2]]}, {Personagens[P[3]]}"
    eventos = f"{Eventos[E[0]]}, {Eventos[E[1]]}, {Eventos[E[2]]}, {Eventos[E[3]]}"
    return lugar, personagens, eventos

In [7]:
# Função que divide um texto em linhas menores, sem cortar palavras no meio.
import textwrap

def dividir_texto_em_linhas(texto, largura=60):
    """
    Divide um texto longo em linhas menores, sem cortar palavras no meio.

    Parâmetros:
        texto (str): O texto a ser dividido.
        largura (int): A largura máxima de cada linha. Padrão é 60 caracteres.

    Retorna:
        str: O texto formatado com quebras de linha.
    """
    # Divide o texto em linhas respeitando o limite de largura
    linhas = textwrap.wrap(texto, width=largura)
    
    # Junta as linhas com quebras de linha
    return "\n".join(linhas)

##### Ollama RPG

Criação da Introdução do cenário

In [65]:
cenario = cenario_aleatorio() # Seleciona o cenário e prepara o prompt
system_prompt = """Você é um criador de narrativas experiente. Sua tarefa é criar narrativas **consistente e coerente**, com base nas instruções fornecidas em 4 mensagens."""

promtpt_introducao = f"""Com base no cenario fornecido, crie um título e uma introdução épica de RPG que seja consistente e coerente, utilizando narrativa imersiva, descritiva, visual e objetiva.  
## Cenario:
{cenario}

## Instruções detalhadas:
1. Forneça nomes para lugares e personagens importantes.  
    - Até 4 personagens, incluindo um antagonista.
    - Entre 3 e 4 lugares, incluindo um local para o desfecho.
2. Conecte de forma lógica o que aconteceu no mundo com o que precisa ser feito e as consequências de falhar.  
3. Use uma narrativa imersiva, descritiva, visual e objetiva para criar um cenário épico e emocionante.  
4. Expanda a ambientação apresentada, incluindo detalhes como vilas, reinos, clima, cultura, povos, lugares, etc.  
5. Sempre que for necessário criar, destruir, recuperar, encontrar algum objeto, limite a no máximo 1 objeto. 

### Atente para os seguintes pontos:  
1. Gênero: Tipo de fantasia em que a aventura se passa.  
2. Algo aconteceu: O que ocorreu no mundo para que ele tenha chegado ao estado atual.  
3. Você precisa: O que precisa ser feito para resolver o estado atual do mundo (no máximo 1 ação ou objetivo).  
4. Senão: As consequências de falhar.  
5. Ambientação: Descrição do mundo em que a aventura se passa.  

## Saida
"""

introducao_schema = (
    "Você deve responder em formato JSON seguindo o seguinte esquema:\n"
    "{\n"
    "  \"title\": \"Título do cenário\",\n"
    "  \"messages\": [\n"
    "    {\"number\": 1, \"content\": \"Descreva detalhadamente a ambientação do mundo (vilas, reinos, clima, cultura, povos, lugares) sem mensionar o que ocorrerá no futuro ou quem fará a mudança. Sempre iniciando com: 'Em um mundo onde ...'.\"},\n"
    "    {\"number\": 2, \"content\": \"Deve apenas descrever como o mundo está **agora**, destacando o que mudou, e explicar em detalhes o evento ou personagem que levou o mundo a esse estado. Sempre iniciando com: 'Mas agora, ...'\"},\n"
    "    {\"number\": 3, \"content\": \"Descreva apenas a ação necessária para resolver o estado atual, com no máximo 1 ação ou objetivo. Não inclua consequências do fracasso nem quem deve executar a ação. Sempre comece com: 'É necessário ...'\"},\n"
    "    {\"number\": 4, \"content\": \"Deve explicar as consequências caso o objetivo não seja cumprido, ligando ela **ao que aconteceu**.\"}\n"
    "  ],\n"
    "  \"personagens\": [\n"
    "    {\"name\": \"Nome dos personagens inimigos ou aliados.(até 4 personagens)\", \"role\": \"Função ou papel do personagem.'\"},\n"
    "  ],\n"
    "  \"locations\": [\n"
    "    {\"name\": \"Nome dos locais omde ocorrem os eventos.(ate 4 locais)\", \"purpose\": \"Função ou propósito do local.'\", \"evento\": \"eventos que ocorrem neste local\"},\n"
    "  ],\n"
    "  \"objective\": \"Informe brevemente o objetivo principal, começando com: 'O objetivo principal é ...'\",\n"
    "}"
)

class Message(BaseModel):
    number: int
    content: str

class Personagem(BaseModel):
    name: str
    role: str

class Location(BaseModel):
    name: str
    purpose: str
    evento: str

# Classe que representa o cenário completo
class Scenario(BaseModel):
    title: str
    messages: List[Message]
    personagens: List[Personagem]
    locations: List[Location]
    objective: str

# O que fazer se o modelo responder com menos de 4 mensagens

In [66]:
introducao = model_response(system_prompt, promtpt_introducao, introducao_schema, Scenario)

print(cenario)

if True:
    print(f"\nTitulo: {introducao.title}")
    print()
    for mensagem in introducao.messages:
        texto = html.unescape(mensagem.content)
        print(dividir_texto_em_linhas(texto, largura=100))
        print("=")
    print(f"Objetivo pincipal: {introducao.objective}")
    print(f"\nPersonagens:")
    for personagem in introducao.personagens:
        print(f"  - {personagem.name}: {personagem.role}")
    print(f"\nLugares:")
    for local in introducao.locations:
        print(f"""  - {local.name}:\n       {local.purpose}\n       {local.evento}""")

Descrisão do Cenário:
- Gênero: Fantasy
- Algo aconteceu: Um Rei bondoso foi assassinado.
- Você precisa: Reunir um grande exército. 
- Senão: O caos tomará o mundo.
- Ambientação: Você é um Aventureiro que se aventura em um incrível mundo de fantasia, rastejando em perigosas masmorras e enfrentando terríveis inimigos!

Titulo: O Reino dos Espelhos

Em um mundo onde a natureza é uma tapeçaria de cores vibrantes, onde os reinos são construídos sobre
colinas de cristal e as vilas são ilhadas em lagos de água cristalina. A cultura é marcada pela
admiração pelos feitos dos heróis do passado e a religião é centrada na crença de que o universo tem
um equilíbrio divino. Os povos vivem em harmonia com a natureza, desenvolvendo tecnologias naturais
e cultivando a terra com sabedoria. A região é dividida em três reinos principais: Eldrador,
conhecido por suas cidades de cristal; Valtor, um território selvagem e perigoso; e Calonia, uma
terra de florestas densas e rios serpentes.
=
Mas agora, o r

In [68]:
system_prompt = "Você é um especialista em criação de histórias para RPG."

personagens = ""
for personagem in introducao.personagens:
   personagens += f"* {personagem.name}: {personagem.role}\n"

nomes = ""
for personagem in introducao.personagens:
   nomes += f"{personagem.name}; "

promtpt = f"""Para todos os personagens {nomes}, escolha o papel de cada personagem de acordo com a sua importância na história.

## Introdução:
1. **Ambientação**:   
{introducao.messages[0].content}

2. **O que aconteceu**: 
{introducao.messages[1].content}

3. **Como resolver**: 
{introducao.messages[2].content}

4. **Consequências de Falhar**:  
{introducao.messages[3].content}

5. **Personagens**:
{personagens}

## Instruções:
1. Aliados, personagem que ajuda os jogadores diretamente na missão. Inclua: Nome, Papel, Motivação e Local de origem que justifique sua utilidade.
2. Antagonista principal, personagem que seja o principal inimigo dos jogadores. Inclua: Nome, Papel, Motivação para se opor e Local de origem que explique seu poder ou influência.
3. Informantes ou NPCs Neutros, personagem que forneça informações ou itens úteis sem alinhar-se totalmente aos jogadores. Inclua: Nome, Papel, Motivação para interagir e Local de origem que justifique seu conhecimento ou recursos. 
4. Personagens Secundários com Dilemas, personagem com um dilema moral ou escolha difícil que impacte a narrativa. Inclua: Nome, Papel, Motivação e Local de origem que destaque o impacto do problema em sua vida.
5. Figuras de Autoridade, personagem influente que pressione ou apoie os jogadores. Inclua: Nome, Papel, Motivação para garantir o sucesso da missão e Local de origem que explique sua liderança.
6. Defina em qual cena o personagem aparece com base no papel narrativo:
    - Introdução: Escolha esta cena para personagens que introduzem o contexto, o conflito inicial, ou que fornecem informações cruciais para iniciar a aventura. Exemplos: líderes locais, testemunhas do problema ou mensageiros.
    - Desenvolvimento: Escolha esta cena para personagens que avançam a trama inicial, como aliados que oferecem suporte, inimigos que complicam o progresso, ou figuras neutras que apresentam desafios sociais ou exploratórios.
    - Clímax Preparatório: Escolha esta cena para personagens que aumentam a tensão ou preparam o caminho para o clímax, como inimigos mais fortes, aliados em perigo, ou figuras que revelam novas informações importantes.
    - Desfecho: Escolha esta cena para personagens essenciais ao confronto final ou à resolução, como o antagonista principal, aliados cruciais ou aqueles que desempenham papéis decisivos no resultado da história.
7. O antagonista principal deve aparecer pelo menos na Cena de Desfecho.
8. Os Personagens {nomes} devem estar presentes obrigatoriamente, mas novos personagenspodem ser criados, se necessário.


**Critério Técnico**: Baseie sua decisão na importância do papel do personagem para a cena específica, garantindo que ele contribua diretamente para o objetivo narrativo daquela etapa.

## Saída Esperada:
"""

personagens_schema = (
"Você deve responder em formato JSON seguindo o seguinte esquema:\n"
"{\n"
"  \"characters\": [\n"
"    {\n"
"      \"name\": \"informe apenas o nome marcante para o personagem.\",\n"
"      \"role\": \"Um papel claro, podendo ser um aliado, antagonista, informante, NPCs Neutro, Personagem Secundário com Dilemas, Figuras de Autoridade, ou figura de autoridade.\",\n"
"      \"motivation\": \"Uma motivação clara, com base no seu papel na história.\",\n"
"      \"place_origin\": \"Nome do local de origem que justifique suas habilidades ou conhecimentos.\",\n"
"      \"scenes\": [\n"
"        {\"name\": \"Em qual cena o personagem aparece: Introdução, Desenvolvimento, Clímax Preparatório, Desfecho\", \"purpose\": \"Qual o objetivo do personagem nessa cena\"}\n"
"      ]\n"
"    }\n"
"  ]\n"
"}"
)

class Scene(BaseModel):
    name: str
    purpose: str

class Character(BaseModel):
    name: str
    role: str
    motivation: str
    place_origin: str
    scenes: List[Scene]

class Characters(BaseModel):
    characters: List[Character]

res = model_response(system_prompt, promtpt, personagens_schema, Characters)
print(f"Total de personagens: {len(res.characters)} - {nomes}")
for character in res.characters:
    print(f">> {character.name}:")
    print(f"    - Papel: {character.role}")
    print(f"    - Motivação: {character.motivation}")
    print(f"    - Local Origem: {character.place_origin}")
    print(f"    - Cenas:")
    for cena in character.scenes:
        print(f"         - {cena.name}: {cena.purpose}")

Total de personagens: 4 - Malakai, o Líder dos Assassinos; Eira, a Herdeira do Trono; 
>> Malakai:
    - Papel: Líder dos Assassinos
    - Motivação: Busca por poder e controle sobre o mundo.
    - Local Origem: Valtor, território selvagem e perigoso
    - Cenas:
         - Desfecho: Confronto final com Eira e defesa do seu reino
         - Introdução: Presentação como o principal antagonista da história
>> Eira:
    - Papel: Herdeira do Trono
    - Motivação: Proteger seu reino e seus habitantes de qualquer ameaça.
    - Local Origem: Eldrador, reino conhecido por suas cidades de cristal
    - Cenas:
         - Desfecho: Confronto final com Malakai para defender o seu reino
         - Introdução: Presentação como a protagonista da história
>> Líder dos Guardas do Reino:
    - Papel: Aliado de Eira, fornece apoio militar na defesa do reino.
    - Motivação: Proteger o seu reino e seus habitantes
    - Local Origem: Eldrador
    - Cenas:
         - Desfecho: Participação no confronto fi

In [ ]:
system_prompt = "Você é um especialista em criação de histórias para RPG."

lugares = ""
for local in introducao.locations:
   lugares += f"* {local.name} ({local.purpose}): {local.evento}\n"

nome_locais = ""
for local in introducao.locations:
   nome_locais += f"{local.name}; "

promtpt = f"""Para cada um dos seguintes locais: {nome_locais}, defina as seguintes informações detalhadas:

## Introdução:
1. **Ambientação**:   
{introducao.messages[0].content}

2. **O que aconteceu**: 
{introducao.messages[1].content}

3. **Como resolver**: 
{introducao.messages[2].content}

4. **Consequências de Falhar**:  
{introducao.messages[3].content}

5. **Personagens**:
{lugares}

## Instruções:
1. **Descrição do Local** inclua elementos visuais (o que se vê), auditivos (sons predominantes) e atmosféricos (sensação ou emoção gerada pelo ambiente).
2. **Papel do Local na Aventura**, explique sua importância narrativa e como ele contribui para a progressão da história.
3. Em quais cenas ele será usado:
    - Introdução: Escolha esta cena para locais que apresentam o contexto inicial, o conflito ou informações essenciais para começar a aventura.
    - Desenvolvimento: Escolha esta cena para locais que ajudam a avançar a trama inicial, como lugares que abrigam aliados, inimigos ou desafios importantes.
    - Clímax Preparatório: Escolha esta cena para locais que aumentam a tensão ou preparam os heróis para o confronto final.
    - Desfecho: Escolha esta cena para o local do confronto final ou da resolução da aventura. Certifique-se de que pelo menos um dos locais seja designado para esta cena.
4. **Objetivo do Local em Cada Cena** explique como o local é usado em cada cena escolhida, destacando sua relevância para o objetivo dos heróis.".
5. Certifique-se de que um dos locais seja escolhido para a cena do Desfecho, e defina como ele contribuirá diretamente para a resolução do conflito principal.

**Critério Técnico**: Baseie sua decisão na importância do local para a cena específica, garantindo que ele contribua diretamente para o objetivo narrativo daquela etapa.

## Saída Esperada:
"""

locations_schema = (
"Você deve responder em formato JSON seguindo o seguinte esquema:\n"
"{\n"
"  \"locations\": [\n"
"    {\n"
"      \"name\": \"Informe apenas o nome marcante para o local.\",\n"
"      \"role\": \"Um papel claro para o local para a aventura.\",\n"
"      \"description\": \"Descrição detalhada, com elementos visuais, auditivos e atmosféricos do local\",\n"
"      \"scenes\": [\n"
"        {\"name\": \"Em qual cena o local aparece: Introdução, Desenvolvimento, Clímax Preparatório, Desfecho. (maximo 2)\", \"purpose\": \"Uma explicação clara, sobre para qual objetivo ele será usado nessa cena\"}\n"
"      ]\n"
"    }\n"
"  ]\n"
"}"
)

class Scene(BaseModel):
    name: str
    purpose: str

class Location(BaseModel):
    name: str
    role: str
    description: str
    scenes: List[Scene]

class Locations(BaseModel):
    locations: List[Location]

res = model_response(system_prompt, promtpt, locations_schema, Locations)
print(f"Total de lugares: {len(res.locations)} - {nome_locais}")

for local in res.locations:
    print(f"{local.name}")
    print(f"    - Papel: {local.role}")
    print(f"    - Descrição: {local.description}")
    print(f"    - Cenas:")
    for cena in local.scenes:
        print(f"         - {cena.name}: {cena.purpose}")

Par cada cena definir o local ao inves de para cada local definir a cena, na esprança de garantir que todas as cenas tenha ao menos um local.

Total de lugares: 3 - Eldrador; Valtor; Calonia; 
Eldrador
    - Papel: Cidade de cristal e centro de poder
    - Descrição: Uma cidade de cristal, com colinas altas que se estendem por todo o lado. O som do choro dos pássaros é constante, enquanto as luzes das lanternas acesas dançam pelas ruas. A atmosfera é de tristeza e desorientação, com os habitantes em choque e desorientados pelo assassinato do rei.
    - Cenas:
         - Introdução: Apresentar o contexto inicial
         - Desenvolvimento: Aproximar os heróis dos assassinos
Valtor
    - Papel: Território selvagem e perigoso
    - Descrição: Um território de densas florestas, com rios serpentes que cortam profundamente a terra. O som dos pássaros é mais agressivo aqui, enquanto as sombras das árvores parecem se mover por conta própria. A atmosfera é de perigo e desconfiança.
    - Cenas:
         - Desenvolvimento: Encontrar aliados para a jornada
         - Clímax Preparatório: Preparar os heróis para o confronto final
Calonia

Divisão dos arcos da aventura

In [168]:
# Prepara os pontos basicos dos 4 arcos
lugares = ""
for local in introducao.locations:
   lugares += f"* {local.name} ({local.purpose}): {local.evento}\n"

personagens = ""
for personagem in introducao.personagens:
   personagens += f"* {personagem.name}: {personagem.role}\n"


prompt_arco = f"""Sua tarefa é ajudar a criar narrativas divididas em 4 cenas claros e progressivos, utilizando os dados fornecidos em **Introdução (Dividida em Partes)** e seguindo as diretrizes a seguir:


## Introdução (Dividida em Partes): 
1. **Ambientação**:   
{introducao.messages[0].content}

2. **O que aconteceu**: 
{introducao.messages[1].content}

3. **Como resolver**: 
{introducao.messages[2].content}

4. **Consequências de Falhar**:  
{introducao.messages[3].content}

5. **Objetivo principal**:  
{introducao.objective}

## Locais Importantes:
{lugares}  

## Personagens Importantes:
{personagens}  

## Instruções:  
1. **Seleção de Locais**: Para cada cena selecione o loclal onde ela ira ocorrer. 
   - Priorize os **Locais Importantes** .  
   - Varie os tipos de locais entre os cenas para criar uma progressão interessante.  

2. **Seleção de Personagens**: Para cada cena selecione um ou mais personagem que irá participar.  
   - Priorize os **Personagens Importantes**. Caso não haja personagens definidos, crie personagens que façam sentido na história. 

3. **Seleção de Eventos**: Para cada cena selecione um evento que ocorra nesse local. 
   - Priorize os evento sugeridos para os locais. Caso não haja escolha eventos que promovam tensão narrativa e reflitam a progressão do arco.  
   - Certifique-se de que os eventos conectem-se diretamente ao objetivo principal.  
   - Relacione os eventos às partes da introdução, conforme indicado abaixo.  

4. **Definição de Objetivos**: Para cada cena defina um objetivo específico. Que seja parte do **objetivo principal**.  
   - Divida o objetivo principal em etapas claras e progressivas ao longo dos 4 cenas.  
   - Cada objetivo deve levar à próxima fase da história, culminando na resolução da trama no arco final.  
   - Relacione os objetivos ao local, personagem e evento do arco.  

5. **Conexão Geral**: Conecte as escolhas para criar um narrativa coesa. 
   - Certifique-se de que as escolhas de locais, personagens, eventos e objetivos conectem-se logicamente e criem uma narrativa coesa. 
   - Certifique-se de que cada arco introduza elementos ou personagens que contribuam diretamente para os eventos ou resoluções nos cenas seguintes, criando uma progressão temática clara e coesa.
   - Escale a tensão e a complexidade dos cenas à medida que a história avança, culminando em um clímax na Cena 4.  

## Saída Esperada:  
"""

arcos_schema = (
"Você deve responder em formato JSON seguindo o seguinte esquema:\n"
"{\n"
"  \"cenas\": [\n"
"    {\n"
"      \"name\": \"Cena 1: Introdução\",\n"
"      \"location\": \"Defina o local onde a cena inicial ocorre, refletindo a ambientação descrita na introdução.\",\n"
"      \"character\": \"Liste os personagens principais apresentados nesse momento, geralmente aqueles que introduzem o conflito ou contexto.\",\n"
"      \"event\": \"Descreva o cenário inicial e o incidente que dá início à aventura, alinhando-se com a 'Ambientação' e 'O que aconteceu' na introdução.\",\n"
"      \"objective\": \"Apresente o primeiro desafio ou mistério que os jogadores precisam compreender para seguir na história.\",\n"
"    },\n"
"    {\n"
"      \"name\": \"Cena 2: Obstáculo Inicial\",\n"
"      \"location\": \"Defina o local onde ocorre o primeiro grande desafio, conectado ao objetivo de encontrar o artefato ou avançar na missão.\",\n"
"      \"character\": \"Inclua aliados ou inimigos que os jogadores encontram, com foco em desenvolver a narrativa inicial.\",\n"
"      \"event\": \"Explique o primeiro grande obstáculo e como ele reflete os passos iniciais para resolver o problema principal, baseado em 'Como resolver'.\",\n"
"      \"objective\": \"Estabeleça o que os jogadores precisam superar nesse arco para seguir na história, como encontrar informações ou superar uma ameaça inicial.\"\n"
"    },\n"
"    {\n"
"      \"name\": \"Cena 3: Escalada\",\n"
"      \"location\": \"Descreva o local onde os eventos se tornam mais complexos ou perigosos, refletindo a progressão da história.\",\n"
"      \"character\": \"Apresente personagens que aumentam a tensão da trama, como inimigos mais poderosos ou aliados em risco.\",\n"
"      \"event\": \"Detalhe como o conflito se intensifica, ligando aos elementos de 'Como resolver' e 'Consequências de Falhar'.\",\n"
"      \"objective\": \"Defina o que os jogadores precisam realizar para evitar grandes perdas e preparar-se para o clímax da aventura.\",\n"
"    },\n"
"    {\n"
"      \"name\": \"Cena 4: Clímax e Resolução\",\n"
"      \"location\": \"Descreva o local onde acontece o confronto final ou a resolução da história, conectando ao objetivo principal.\",\n"
"      \"character\": \"Inclua o antagonista principal ou aliados cruciais que desempenham papéis essenciais no desfecho.\",\n"
"      \"event\": \"Explique o confronto ou evento final, com foco nas 'Consequências de Falhar' e no 'Objetivo principal'.\",\n"
"      \"objective\": \"Estabeleça o que os jogadores precisam alcançar para concluir a aventura com sucesso, como derrotar o vilão ou proteger o artefato.\",\n"
"    }\n"
"  ]\n"
"}"
)

class Cena(BaseModel):
   name: str
   location: str
   character: str
   event: str
   objective: str

class Cenas(BaseModel):
   cenas: List[Cena]


system_prompt = """Você é um especialista em criação de histórias para RPG. Siga as instruções fornecidas e responda em formato JSON seguindo o esquema especificado."""

In [169]:
base_arcos = model_response(system_prompt, prompt_arco, arcos_schema, Cenas)
base_arcos.model_dump()

{'cenas': [{'name': 'Cena 1: Introdução',
   'location': 'Defina o local onde a cena inicial ocorre, refletindo a ambientação descrita na introdução.',
   'character': 'Liste os personagens principais apresentados nesse momento, geralmente aqueles que introduzem o conflito ou contexto.',
   'event': "Descreva o cenário inicial e o incidente que dá início à aventura, alinhando-se com a 'Ambientação' e 'O que aconteceu' na introdução.",
   'objective': 'Apresente o primeiro desafio ou mistério que os jogadores precisam compreender para seguir na história.'},
  {'name': 'Cena 2: Obstáculo Inicial',
   'location': 'Defina o local onde ocorre o primeiro grande desafio, conectado ao objetivo de encontrar o artefato ou avançar na missão.',
   'character': 'Inclua aliados ou inimigos que os jogadores encontram, com foco em desenvolver a narrativa inicial.',
   'event': "Explique o primeiro grande obstáculo e como ele reflete os passos iniciais para resolver o problema principal, baseado em 'Co

In [170]:
print(prompt_arco)

Sua tarefa é ajudar a criar narrativas divididas em 4 cenas claros e progressivos, utilizando os dados fornecidos em **Introdução (Dividida em Partes)** e seguindo as diretrizes a seguir:


## Introdução (Dividida em Partes): 
1. **Ambientação**:   
Em um mundo onde a natureza parece ter perdido seu equilíbrio, os reinos de Eldrador e Valtoria se estendem como faróis no céu noturno. As montanhas majestosas de Eldrador oferecem abrigo às tribos nômades que vivem em harmonia com a terra, enquanto as florestas mágicas de Valtoria escondem segredos antigos. As cidades-estado de Calonia e Arkeia brilham como joias no deserto, suas torres de cristal refletindo a luz do sol ao pôr-do-sol. A cultura é rica e diversificada, com uma mistura única de tradições e influências estrangeiras.

2. **O que aconteceu**: 
Mas agora, o mundo está mudo. Um exército de mortos avança em direção às cidades-estado, seu barulho ecoa pelas montanhas e florestas como um rugido de fera. A magia que mantinha os mort

Criação das cenas

In [128]:
## Cena 1: Introdução e Contexto
introducao_completa = texto_introducao
local_evento_atual = "Uma pequena vila e envolve Um grande mistério"
proximo_local_evento = "Um grande reino e envolve O Vilão ataca"
# {local} e envolve {evento}

prompt_sequencia_narracao = f"""### Contexto da Aventura
{introducao_completa}

### Tipo de Cena
Esta cena apresenta o cenário, o contexto e o gancho principal da história. Seu objetivo é colocar o personagem em movimento, mostrando a crise que ele deve resolver e respondendo a perguntas como: O que está em jogo? Quem ou o que causa o problema? Qual é o objetivo inicial do personagem?  

### Instruções 
1. Priorize **locais, eventos e personagens** mencionados na introdução.  
2. Caso a introdução não forneça informações suficientes, utilize as sugestões abaixo como fallback:  
   - **Local  e Evento Atual**: {local_evento_atual}  
   - **Próximo Local e Evento**: {proximo_local_evento} 
3. **Twists e Obstáculos**: Sugira até dois twists ou obstáculos inesperados para aumentar a tensão e manter o interesse.
4. Seja consistente e coerente ao definir o objetivo da cena atual e o objetivo para o proximo local.
5. **Extração de Informações** extraia exclusivaemente do Contexto da Aventura os seguintes dados:
  - **Itens importantes**: Informe o nome, função e condição de uso para os itens importantes para a trama principal e o seu status: "Não encontrado", "Não utilizado" e "Usado".
  - **Personagens importantes**: Informe o nome, papel e motivações para os personagens para a trama principal e o se sua participação ja foi concluida: "Não apareceu", "Não concluido" e "Concluido".
  - **Assuntos importantes**: Informe o nome, a da descrisão dos assuntos importantes para a trama principal e status se ele ja foi ou não tratado: "Não resonvido", "resonvido".

**Nota**: Sempre siga o tipo de cena descrito no contexto para manter o foco narrativo correto.
### Saida
"""
narrativa_schema = (
    "Você deve responder em formato JSON seguindo o seguinte esquema:\n"
    "{\n"
    "  \"name_locale\": \"Nome do local\",\n"
    "  \"scene_description\": \"Descreva o local com detalhes sensoriais e apresente o evento principal. Foque em criar atmosfera e introduzir conflitos ou desafios.\",\n"
    "  \"objetivo_description\": \"Explique o objetivo do personagem com uma ação iniciada no verbo no infinitivo\",\n"
    "  \"Connection_next_location\": \"Indique o próximo local no local atual com uma ação no imperativo e introduza um objetivo para o próximo local\",\n"
    "  \"itens\": [\n"
    "    {\"name\": \"Nome do item\", \"function\": \"Função do item\", \"condition_use\": \"Condição de uso do item\", \"status\": \"Status do item\"}\n"
    "  ],\n"
    "  \"personagens\": [\n"
    "    {\"name\": \"Nome do personagem\", \"role\": \"Papel do personagem\", \"motivation\": \"Motivação/segredos do personagem\", \"participation\": \"Seu participação ja foi concluida\"}\n"
    "  ],\n"
    "  \"assuntos\": [\n"
    "    {\"name\": \"Nome para o assunto\", \"description\": \"Breve explicação sobre o assunto\", \"status\": \"Status do assunto\"}\n"
    "  ],\n"
    "  \"twists\": [\n"
    "    {\"description\": \"Descrição da reviravolta\"}\n"
    "  ]\n"
    "}"
)

In [107]:
# Cena 2: Continuação da Cena Anterior
introducao_completa = texto_introducao
objetivo_anterior = "Investigar as mortes e desaparecimentos" # Objetivo da cena anterior (foi concuido)
local_evento_atual = "Partir em direção às Montanhas de Ardesia em busca dos Símbolos da Divindade" #(novo objetivo)


prompt_sequencia_narracao = f"""### Contexto da Aventura
{introducao_completa}

### Próxima Etapa da Jornada
Após {objetivo_anterior}, a história avança para {local_evento_atual}, onde um novo desafio ou descoberta impede o personagem de alcançar seu objetivo diretamente.

### Instruções
1. Respeite os **objetivos definidos**:
   - O **objetivo atual** deve ser cumprido **neste local** e não deve ser alterado ou substituído.
   - A **conexão com o próximo local** deve apresentar uma **ação nova** que avança a trama, sem repetir o objetivo atual.
   - A conexão com o próximo local deve ser independente e não pode interferir ou impedir a conclusão do objetivo atual.
2. **Local e Evento**:
   - O local deve ser descrito com **detalhes sensoriais** (visuais, sons, cheiros) para criar uma atmosfera imersiva.
   - O evento principal deve introduzir personagens, obstáculos ou conflitos que desafiam o personagem a cumprir seu objetivo.
3. **Twists e Obstáculos**:
   - Sugira até dois twists ou obstáculos inesperados que dificultem ou compliquem a jornada do personagem.

**Nota**: Sempre siga o tipo de cena descrito no contexto para manter o foco narrativo correto.
### Saida
"""

narrativa_schema = (
   "Você deve responder em formato JSON seguindo o seguinte esquema:\n"
   "{\n"
   "  \"name_locale\":  \"Nome do local\",\n"
   "  \"scene_description\": \"Descreva o local com detalhes sensoriais e apresente o evento principal. Foque em criar atmosfera e introduzir conflitos ou desafios.\"\n,"
   "  \"objetivo_description\": \"Descreva o objetivo do personagem no local atual, usando o verbo no infinitivo. NÃO mude o objetivo fornecido.\"\n,"
   "  \"Connection_next_location\": \"Apresente uma ação nova para o próximo local, evitando repetir o objetivo atual. A conexão deve avançar a trama com um novo desafio ou descoberta.\"\n,"
   "  \"twists\": [\n"
   "    {\n"
   "      \"description\": \"Descrição da reviravolta\"\n"
   "    },\n"
   "  ]\n"
   "}"
)

In [110]:
# Cena 3: Continuação da Cena Anterior para o clímax
introducao_completa = texto_introducao
objetivo_anterior = "Investigar o local e encontrar pistas sobre os Símbolos da Divindade" # Objetivo da cena anterior (foi concuido)
local_evento_atual = "Solicitar informações à figura encapuzada sobre os símbolos" #(novo objetivo)


prompt_sequencia_narracao = f"""### Contexto da Aventura
{introducao_completa}

### Próxima Etapa da Jornada
- Após cumprir o objetivo anterior: **{objetivo_anterior}**, o personagem agora deve avançar para: **{local_evento_atual}**.
- Esta cena deve preparar a transição direta para o **clímax da trama principal**, que será resolvido na próxima etapa.

### Instruções
1. **Objetivo da Cena**:
   - O personagem deve cumprir o **objetivo definido** nesta cena, sem modificá-lo ou desviá-lo: {local_evento_atual}.
   - A ação nesta cena deve ser **essencial** para avançar a história em direção ao clímax.

2. **Descrição do Local e Evento**:
   - Descreva o local com **detalhes sensoriais ricos** (visuais, sons, cheiros) para criar uma atmosfera imersiva.
   - Introduza um **evento principal** com personagens ou obstáculos que desafiem o personagem a cumprir o objetivo.

3. **Conexão com o Clímax**:
   - Apresente o **próximo local** com uma ação clara que leve diretamente ao **clímax da aventura**.
   - A conexão deve ser lógica e preparar a resolução da trama principal.
   - A conexão com a proxima cena deve ser diferente do objetivo anterior e objetivo atual

4. **Twists e Obstáculos**:
   - Sugira até **dois twists** ou obstáculos inesperados que aumentem a tensão, mas sem impedir a conclusão do objetivo atual.

**Nota**: Sempre siga o tipo de cena descrito no contexto para manter o foco narrativo correto.
### Saida
"""



narrativa_schema = (
   "Você deve responder em formato JSON seguindo o seguinte esquema:\n"
   "{\n"
   "  \"name_locale\":  \"Nome do local\",\n"
   "  \"scene_description\": \"Descreva o local com detalhes sensoriais e apresente o evento principal. Foque em criar atmosfera e introduzir conflitos ou desafios.\"\n,"
   "  \"objetivo_description\": \"Descreva o objetivo do personagem no local atual, usando o verbo no infinitivo. NÃO mude o objetivo fornecido.\"\n,"
   "  \"Connection_next_location\": \"Indique o próximo local com uma ação no imperativo e e conecte diretamente ao clímax da trama principal\"\n,"
   "  \"twists\": [\n"
   "    {\n"
   "      \"description\": \"Descrição da reviravolta\"\n"
   "    },\n"
   "  ]\n"
   "}"
)

In [114]:
# Cena 4: Final da Aventura
introducao_completa = texto_introducao
local_evento_atual = "Subir as escadas para encontrar o encontro com a figura encapuzada"
# {local} e envolve {evento}

prompt_sequencia_narracao = f"""### Contexto da Aventura
{introducao_completa}

### Cena Final: Desfecho da Aventura
Esta é a última cena da história, onde o personagem encara seu maior desafio e resolve (ou não) o problema central de forma definitiva. Não haverá novas cenas ou locais. O foco está no **clímax** e no encerramento da trama.

### Evento Final
A história chega ao seu ápice em {local_evento_atual}, onde o personagem enfrenta seu maior desafio. 
Esta é a última chance de resolver o problema principal de forma definitiva, determinando o destino da jornada e suas consequências.

### Instruções
1. **Local Atual**: Descreva o cenário com **detalhes sensoriais ricos** (visuais, sons, cheiros) para criar uma atmosfera tensa e imersiva, adequada ao confronto final.
2. **Evento Principal**: Apresente o **desafio decisivo** que o personagem enfrenta neste local. Inclua personagens envolvidos e a situação que leva ao desfecho.
3. **Objetivo do Personagem**: Descreva a **ação final** que o personagem deve realizar para resolver o conflito principal. Inicie a frase com o verbo no infinitivo.
4. **Conclusão e Consequências**: Resolva a história e explique as **consequências** das ações do personagem. Deixe claro se ele **alcança seu objetivo** ou não e quais são os impactos finais na trama.
5. **Twists Finais**: Sugira até **dois twists** ou revelações surpreendentes para tornar o final mais interessante, mas que não abram novos caminhos para a história.

**Importante**: 
- Esta é a **última cena** da aventura, e a história deve **terminar aqui**. Não deve haver conexão com um próximo local ou evento.
- O desfecho precisa resolver todos os pontos principais da trama.

### Saída
"""
narrativa_schema = (
   "Você deve responder em formato JSON seguindo o seguinte esquema:\n"
   "{\n"
   "  \"name_locale\":  \"Nome do local\",\n"
   "  \"scene_description\": \"Descreva o local com detalhes sensoriais e apresente o evento principal. Foque em criar atmosfera e introduzir conflitos ou desafios.\"\n,"
   "  \"objetivo_description\": \"Descreva o objetivo do personagem no local atual, usando o verbo no infinitivo. NÃO mude o objetivo fornecido.\"\n,"
   "  \"Connection_next_location\": \"Explique como a história termina e as consequências das ações do personagem.\"\n," # Na hora de passar isso para o modelo, isso deve ser chamado de concluisão. Não mudei o nome aui só para não precisar ceiar uma noca classe.
   "  \"twists\": [\n"
   "    {\n"
   "      \"description\": \"Descrição da reviravolta\"\n"
   "    },\n"
   "  ]\n"
   "}"
)

In [124]:
class Twist(BaseModel):
    description: str

class Personagem(BaseModel):
    name: str
    role: str
    motivation: str
    participation: str

class Item(BaseModel):
    name: str
    function: str
    condition_use: str
    status: str

class Assunto(BaseModel):
    name: str
    description: str
    status: str

class Location(BaseModel):
    name_locale: str
    scene_description: str
    objetivo_description: str
    Connection_next_location: str
    itens: List[Item]
    personagens: List[Personagem]
    assuntos: List[Assunto]
    twists: List[Twist]

In [324]:
# Cena 1
local_cena_1 = base_arcos.arcos[0].local
personagem_cena_1 = base_arcos.arcos[0].personagem
evento_cena_1 = base_arcos.arcos[0].evento
objetivo_cena_1 = base_arcos.arcos[0].objetivos
local_proxima_cena = base_arcos.arcos[1].local
objetivo_proxima_cena = base_arcos.arcos[1].objetivos

prompt_cena = f"""Com base nos dados fornecidos, descreva a cena inicial da aventura de RPG que apresente o mundo, o contexto e motive os jogadores.
## Introdução da história: 
{introducao.messages[0].content} # apenas as duas partes relevantes para a cena 1
{introducao.messages[1].content}

## Local da Cena 1: 
{local_cena_1}

## Personagem da Cena 1:
{personagem_cena_1}

## Evento da Cena 1: 
{evento_cena_1}

## Objetivo da Cena 1:
{objetivo_cena_1}

## Local da Próxima Cena: 
{local_proxima_cena}

## Objetivo da Próxima Cena: (objetivo inicial que leva a esse lugar, ao chegar la pode acontecer alqo que leve a outro objetivo)
{objetivo_proxima_cena}

## Instruções:
1. Apresentar o cenário e o contexto do local atual.
2. Introduzir o personagem relacionado ao local ou à trama.
3. Descrever o evento significativo que ocorre no local, conectando-o ao objetivo principal.
4. Mostrar o gancho narrativo, destacando a crise ou desafio que o personagem deve resolver.
5. Fazer uma ligação natural para a próxima cena, mencionando o próximo local e objetivo.
6. Sugira até dois twists ou obstáculos inesperados para aumentar a tensão e manter o interesse.

## Saída Esperada:  
"""

cena_schema = (
    "Você deve responder em formato JSON seguindo o seguinte esquema:\n"
    "{\n"
    "  \"title\": \"Um titulo para a cena.\",\n"
    "  \"description\": \"Descreva os principais pontos que o Mestre devera usar para contruir a narrativa dessa cena. Um resumo do que pode acontecer do incio ao fim conectando o final dessa cena com o local e objetivo da proxima cena.\",\n"
    "  \"twists\": [\n"
    "    {\"description\": \"Descrição da reviravolta\"}\n"
    "  ]\n"
    "}"
)

class Twist(BaseModel):
    description: str

class Cena(BaseModel):
    title: str
    description: str
    twists: List[Twist]


"""
Cena:
    cena atual:
        resumo do que acontece nessa cena, quem ira fazer o que e onde vai fazer e porque vai fazer.
        o que pode acontecer (se der certo ou errado) e pra onde vai e porque vai ( tiver dado certo ou errado)
    proxima cena:
        local_objetivo: pra onde vai e o porque vai.
    twists:

"""

'\nCena:\n    cena atual:\n        resumo do que acontece nessa cena, quem ira fazer o que e onde vai fazer e porque vai fazer.\n        o que pode acontecer (se der certo ou errado) e pra onde vai e porque vai ( tiver dado certo ou errado)\n    proxima cena:\n        local_objetivo: pra onde vai e o porque vai.\n    twists:\n\n'

In [322]:
system_prompt = """Você é um criador de narrativas para RPG de mesa. Sua tarefa é descrever cenas para ajudar o Mestre de Jogo a narrar a aventura.
1. **Foco**: Apresente o cenário, o evento principal e os personagens envolvidos, conectando-os ao objetivo da cena.
2. **Protagonistas**: Os jogadores são os personagens principais, e os NPCs (personagens fornecidos) estão lá para apoiar ou desafiar a história.
3. **Clareza**: A descrição deve ser direta e evocativa, sem excesso de detalhes, mas suficiente para criar uma cena imersiva.
4. **Conexão**: Faça uma transição natural para a próxima cena, destacando o próximo local e objetivo.

"""

In [ ]:
print(prompt_cena)

In [323]:
base_cena = model_response(system_prompt, prompt_cena, cena_schema, Cena)
base_cena.model_dump()
print(f"Titulo: {base_cena.title}")
print(f"{dividir_texto_em_linhas(base_cena.description, largura=100)}")
print(f"\nTwinst:")
for twinst in base_cena.twists:
    print(f"- {dividir_texto_em_linhas(twinst.description, largura=100)}")

Titulo: Cena Inicial: Luta pela Vida em Eldrador
A cidade-estado de Eldrador está sendo invadida por um exército de mortos. O rei Arin III,
desesperado para salvar sua capital, pede ajuda ao seu melhor alguém, Eira Shadowglow, uma
habilidosa caçadora maga que busca uma fórmula para desbloquear a magia das montanhas de
Kristalberg. Enquanto isso, o exército dos mortos se aproxima e invade Eldrador, matando inocentes e
destruindo tudo em seu caminho.

Twinst:
- Um dos soldados vivos do rei Arin III é visto conversando com um dos líderes do exército dos mortos,
sugerindo que pode haver uma trama mais profunda por trás da invasão.
- Eira descobre um mapa antigo escondido em sua habitação, apontando para as montanhas de Kristalberg
e indicando a localização da fórmula que ela busca.


Define a cena inicial (ve o que da pra simplificar, talvez de pra tirar os exemplos)

In [225]:
cena_inicial_cena = """Você é um chatbot narrador de aventuras de RPG de mesa, que usa o estilo de narração imersivo descritivo para uma descrição detalhada do ambiente, personagem ou evento com detalhes sensoriais (visão, som, cheiro, tato, etc.) e escreve as falas em uma nova linha com o estilo de trancrisção iniciando as falas com um hífen (—).
Crie uma introdução para a cena inicial em até duas mensagens, tendo como base a Introdução e Descrisão da cena.
Gere textos curtos, com poucas palavras.
Sempre finalize com um gancho claro, como: "O que vocês fazem?"
Informe o Status atual da narrativa, que pode ser: "Em andamento", "Finalizada".
Siga o formato das mensagens abaixo para criar a introdução da cena:
Exemplo 1
Mensagem 1: (uma curta descrição da Cena) 
A vila de Grenwald está em silêncio mortal. Telhados afundados, musgo escuro cobrindo as paredes e um cheiro de podridão pairando no ar. Você caminha firme, a armadura rangendo, enquanto observa as marcas de garras nas portas com olhos inquietos. Um vento frio sopra, trazendo uma sensação de que estão sendo observados.

Mensagem 2: (Ação ou fala do NPC)
Do beco, um velho caçador surge da sombra, o arco firme nas mãos. Ele olha diretamente para Você.
— "Não devia estar aqui."
Sua voz rouca ecoa, e ele aponta para as marcas de garras.
— "Ele passou por aqui há muito tempo... mas o mal nunca dorme."
O silêncio pesa. Você segura o cabo da espada. O que vocês fazem?

Exemplo 2: 
Mensagem 1: (uma curta descrição da Cena) 
A taverna "O Cangaceiro Vermelho" vibra com vozes embriagadas, cheiro de cerveja barata e lamparinas de óleo iluminando o ambiente. Vocês observam tudo de um canto, atento, enquanto um de vocês mexe em algumas sacolas deixadas para trás.

Mensagem 2: (Ação ou fala do NPC)
Um estrondo violento! A porta é arrombada, e homens em armaduras invadem a taverna.
— "Todo mundo no chão, agora!" — grita o líder dos invasores.
Clientes correm, cadeiras caem, e o cheiro de ferro e fumaça invade o salão. O que vocês fazem?"""

In [226]:
mensagens_schema = (
    "Você deve responder em formato JSON seguindo o seguinte esquema:\n"
    "{\n"
    "  \"title\": \"Título do cenário\",\n"
    "  \"messages\": [\n"
    "    {\"number\": 1, \"content\": \"Primeira mensagem\"},\n"
    "    {\"number\": 2, \"content\": \"Segunda mensagem\"}\n"
    "  ],\n"
    "  \"status\": \"Status atual da narrativa\"\n"
    "}"
)

In [227]:
class Message(BaseModel):
    number: int
    content: str

# Classe que representa o cenário completo
class Messages(BaseModel):
    title: str
    messages: List[Message]
    status: str

In [228]:
# enables `response_model` in create call
client = instructor.from_openai(
    OpenAI(
        base_url="http://localhost:11434/v1",
        api_key="ollama",  # required, but unused
    ),
    mode=instructor.Mode.JSON,
)


nome_modelo = lista_modelos[modelo_id]
print(f"{nome_modelo}\n")
narrativa = client.chat.completions.create(
    model=nome_modelo,
    messages=[
        {
            "role": "system",
            "content": f"{cena_inicial_cena}\n\n{mensagens_schema}"
        },
        {   "role": "user",
            "content": f"Introdução:\n{texto_introducao[0]}\n{texto_introducao[1]}\n{texto_introducao[2]}\n{texto_introducao[3]}\n\Twist para a aventura:\n{lista_twist}\n\nDescrisão da cena:\n{lista_cenas[0]}"
        }
    ],
    response_model= Messages,
    extra_body={"gpu_layers":-1, "temperature": 0.7},
    
)


if True:
    print(f"{narrativa.title}")
    print()
    for mensagem in narrativa.messages:
        print(dividir_texto_em_linhas(mensagem.content, largura=100))
        print("=")
    print(f"\n{narrativa.status}")
else:
    print(narrativa.choices[0].message.content)

llama3.2:3b-instruct-q8_0
O Início do Despertar

A vila de Início é uma pequena comunidade de casas de madeira e telhados de palha, cercada por uma
floresta densa e escura. A vila está preparada para enfrentar o desafio que se aproxima, com os
moradores armados com machados e lanças. O ar está cheio do cheiro de queimado e a sensação de
inquietude é palpável.
=
De repente, um rugido ecoa pela floresta e o solo começa a tremer. Os moradores da vila olham em
direção à floresta, os olhos wide de medo. Um homem grande e alto surge da floresta, com olhos
vermelhos brilhantes e uma pele escura como carvão. Ele é vestido com armadura negra e carrega um
bastão que emite uma luz azulada.
=

Em andamento
